In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
with open('/content/drive/MyDrive/Anshul/input.txt','r', encoding='UTF-8') as f:
  text=f.read()

In [3]:
chars= sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
stoi={ch:i for i, ch in enumerate(chars)}
itos={i:ch for i, ch in enumerate(chars)}
encode=lambda s: [stoi[c] for c in s]
decode= lambda d: ''.join([itos[c] for c in d])
print(encode('Anshul'))
print(decode(encode('Anshul')))

[13, 52, 57, 46, 59, 50]
Anshul


In [5]:
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [6]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [40]:
torch.manual_seed(1337)
block_size=256
batch_size=64
n_embd=384
n_head=6
n_layer=6
dropout=0.2
max_iters=5000
eval_interval=500
eval_iters=200
learning_rate = 1e-3

def get_batch(split):
  data=train_data if split=='train' else val_data
  ix=torch.randint(len(data)-block_size,(batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb, yb= get_batch('train')
print('inputs:', xb.shape, xb)
print('targets: ', yb.shape, yb)

def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


inputs: torch.Size([64, 256]) tensor([[ 0, 26, 53,  ..., 56, 43, 47],
        [60, 43, 56,  ..., 56,  1, 41],
        [26, 21, 33,  ..., 26, 21, 13],
        ...,
        [ 5, 57,  1,  ...,  1, 35, 47],
        [56, 53, 53,  ..., 59, 50, 42],
        [42, 47, 56,  ..., 39, 56,  1]])
targets:  torch.Size([64, 256]) tensor([[26, 53, 58,  ..., 43, 47, 45],
        [43, 56,  1,  ...,  1, 41, 53],
        [21, 33, 31,  ..., 21, 13, 10],
        ...,
        [57,  1, 52,  ..., 35, 47, 50],
        [53, 53, 58,  ..., 50, 42,  1],
        [47, 56, 43,  ..., 56,  1, 51]])


In [41]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.key=nn.Linear(n_embd, head_size, bias=False)
    self.query=nn.Linear(n_embd, head_size, bias=False)
    self.value=nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout=nn.Dropout(dropout)

  def forward(self, x):
    B,T,C= x.shape
    k=self.key(x)
    q=self.query(x)
    wei=q @ k.transpose(-2,-1) * C**0.5
    wei=wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))
    wei=F.softmax(wei, dim=-1)
    wei=self.dropout(wei)
    v=self.value(x)
    out=wei @ v
    return out

In [42]:
class MultiHeadAttention(nn.Module):

  def __init__(self, num_heads,head_size):
    super().__init__()
    self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj=nn.Linear(n_embd, n_embd)
    self.dropout=nn.Dropout(dropout)

  def forward(self,x):
    out= torch.cat([h(x) for h in self.heads], dim=-1)
    out=self.dropout(self.proj(out))
    return out

In [43]:
class Block(nn.Module):

  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size=n_embd//n_head
    self.sa=MultiHeadAttention(n_head, head_size)
    self.ffwd= FeedForward(n_embd)
    self.ln1=nn.LayerNorm(n_embd)
    self.ln2=nn.LayerNorm(n_embd)


  def forward(self, x):
    x=x+self.sa(self.ln1(x))
    x=x+self.ffwd(self.ln2(x))
    return x

In [44]:
class FeedForward(nn.Module):

  def __init__(self, n_embd):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(n_embd, 4*n_embd),
        nn.ReLU(),
        nn.Linear(4*n_embd, n_embd),
        nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)

In [45]:
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table=nn.Embedding(block_size, n_embd)
    self.blocks=nn.Sequential(
        Block(n_embd,n_head=4),
        Block(n_embd,n_head=4),
        Block(n_embd,n_head=4),
        nn.LayerNorm(n_embd),
    )
    self.lm_head=nn.Linear(n_embd, vocab_size)
    self.ln_f = nn.LayerNorm(n_embd) # final layer norm

  def forward(self, idx, targets=None):
    B,T=idx.shape

    tok_embd=self.token_embedding_table(idx)
    pos_embd=self.position_embedding_table(torch.arange(T))
    x=tok_embd+pos_embd
    x=self.blocks(x)
    x=self.ln_f(x)
    logits=self.lm_head(x)



    if targets==None:
      loss=None
    else:
      B,T,C= logits.shape
      logits=logits.view(B*T, C)
      targets=targets.view(B*T)
      loss=F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx



In [46]:

model = BigramLanguageModel()
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

5.469761 M parameters


In [47]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

KeyboardInterrupt: 